# XAI Analysis: Explaining Higgs Boson Predictions

This notebook applies various Explainable AI (XAI) methods to understand model predictions:
- SHAP (SHapley Additive exPlanations)
- LIME (Local Interpretable Model-agnostic Explanations)
- Integrated Gradients
- DeepLIFT
- Gradient SHAP

In [ ]:
import sys
sys.path.append('../src')

import torch
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from data_loader import load_higgs_data, get_feature_names
from models import create_model
from xai_methods import XAIAnalyzer
from visualization import plot_attribution_heatmap

%matplotlib inline

## Configuration

In [ ]:
# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Configuration
config = {
    'model_type': 'simple',
    'model_path': '../models/higgs_classifier_simple.pth',
    'n_test_samples': 500  # Number of test samples to analyze
}

## Load Data and Model

In [ ]:
# Load data
X_train, X_test, y_train, y_test = load_higgs_data(
    data_path='../data/HIGGS.csv',
    n_samples=50000,
    test_split=0.2,
    random_seed=42
)

feature_names = get_feature_names()

# Use subset for faster XAI computation
X_test_subset = X_test[:config['n_test_samples']]
y_test_subset = y_test[:config['n_test_samples']]

print(f"Training samples: {len(X_train)}")
print(f"Test samples for XAI: {len(X_test_subset)}")
print(f"Number of features: {X_train.shape[1]}")

In [ ]:
# Load model
input_dim = X_train.shape[1]
model = create_model(
    model_type=config['model_type'],
    input_dim=input_dim
)

# Try to load trained weights, otherwise use untrained model
try:
    model.load_state_dict(torch.load(config['model_path'], map_location=device))
    print(f"Loaded model from {config['model_path']}")
except FileNotFoundError:
    print(f"Model file not found at {config['model_path']}")
    print("Using untrained model for demonstration purposes")

model.to(device)
model.eval()

print(f"\nModel parameters: {sum(p.numel() for p in model.parameters())}")

## Initialize XAI Analyzer

In [ ]:
# Create XAI analyzer
xai_analyzer = XAIAnalyzer(
    model=model,
    feature_names=feature_names,
    device=device
)

print("XAI Analyzer initialized")

## 1. SHAP Analysis

SHAP (SHapley Additive exPlanations) provides a unified measure of feature importance based on game theory.

In [ ]:
print("Computing SHAP values...")
shap_values, shap_explainer = xai_analyzer.compute_shap_values(
    X_background=X_train,
    X_test=X_test_subset,
    n_background=100
)

print(f"SHAP values shape: {shap_values.shape}")

In [ ]:
# Plot SHAP summary
xai_analyzer.plot_shap_summary(
    shap_values,
    X_test_subset,
    max_display=20,
    save_path='../figures/shap_summary.png'
)

In [ ]:
# Plot feature importance from SHAP
xai_analyzer.plot_feature_importance(
    shap_values,
    top_k=15,
    title='Feature Importance (SHAP)',
    save_path='../figures/shap_importance.png'
)

## 2. LIME Analysis

LIME explains individual predictions by learning an interpretable model locally around the prediction.

In [ ]:
# Select a few interesting examples
# Get signal and background examples
signal_idx = np.where(y_test_subset == 1)[0][0]
background_idx = np.where(y_test_subset == 0)[0][0]

print(f"Analyzing signal example at index {signal_idx}")
lime_exp_signal = xai_analyzer.compute_lime_explanation(
    X_train=X_train,
    X_instance=X_test_subset[signal_idx],
    num_features=10
)

print("\nTop features for signal prediction:")
for feature, weight in lime_exp_signal['feature_weights']:
    print(f"  {feature}: {weight:.4f}")

In [ ]:
print(f"\nAnalyzing background example at index {background_idx}")
lime_exp_background = xai_analyzer.compute_lime_explanation(
    X_train=X_train,
    X_instance=X_test_subset[background_idx],
    num_features=10
)

print("\nTop features for background prediction:")
for feature, weight in lime_exp_background['feature_weights']:
    print(f"  {feature}: {weight:.4f}")

## 3. Integrated Gradients

Integrated Gradients attributes the prediction to input features by integrating gradients along a path from a baseline.

In [ ]:
print("Computing Integrated Gradients...")
ig_attributions = xai_analyzer.compute_integrated_gradients(
    X_test=X_test_subset,
    n_steps=50
)

print(f"IG attributions shape: {ig_attributions.shape}")

In [ ]:
# Plot feature importance from Integrated Gradients
xai_analyzer.plot_feature_importance(
    ig_attributions,
    top_k=15,
    title='Feature Importance (Integrated Gradients)',
    save_path='../figures/ig_importance.png'
)

## 4. DeepLIFT

DeepLIFT compares the activation of each neuron to its reference activation.

In [ ]:
print("Computing DeepLIFT attributions...")
deeplift_attributions = xai_analyzer.compute_deeplift(
    X_test=X_test_subset
)

print(f"DeepLIFT attributions shape: {deeplift_attributions.shape}")

In [ ]:
# Plot feature importance from DeepLIFT
xai_analyzer.plot_feature_importance(
    deeplift_attributions,
    top_k=15,
    title='Feature Importance (DeepLIFT)',
    save_path='../figures/deeplift_importance.png'
)

## 5. Compare XAI Methods

Compare all XAI methods side by side.

In [ ]:
# Compare methods
comparison_results = xai_analyzer.compare_methods(
    X_background=X_train,
    X_test=X_test_subset[:100],  # Use smaller subset for faster computation
    methods=['shap', 'ig', 'deeplift'],
    save_path='../figures/xai_comparison.png'
)

## 6. Attribution Heatmap

Visualize attributions across multiple samples.

In [ ]:
# Plot attribution heatmap for SHAP values
plot_attribution_heatmap(
    shap_values[:50],  # First 50 samples
    feature_names,
    n_samples=20,
    save_path='../figures/attribution_heatmap.png'
)

## 7. Feature Importance Ranking

Compare feature rankings across different XAI methods.

In [ ]:
import pandas as pd

# Calculate mean absolute attributions for each method
shap_importance = np.abs(shap_values).mean(axis=0)
ig_importance = np.abs(ig_attributions).mean(axis=0)
deeplift_importance = np.abs(deeplift_attributions).mean(axis=0)

# Create DataFrame
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'SHAP': shap_importance,
    'Integrated Gradients': ig_importance,
    'DeepLIFT': deeplift_importance
})

# Sort by SHAP importance
importance_df = importance_df.sort_values('SHAP', ascending=False)

print("\nTop 15 Most Important Features:")
print(importance_df.head(15).to_string(index=False))

## Summary

This notebook demonstrated various XAI methods for explaining Higgs boson classification:

1. **SHAP**: Provides global feature importance and individual explanations
2. **LIME**: Explains individual predictions with local interpretable models
3. **Integrated Gradients**: Attributes predictions through gradient integration
4. **DeepLIFT**: Compares activations to reference values

All methods help understand which features are most important for distinguishing signal from background events.